In [1]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()
db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')
db_host = os.environ.get('DB_HOST')
db_port = os.environ.get('DB_PORT')
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)


In [21]:
class DatabaseManipulation:
    def __init__(self):
        self.conn = conn
        self.cur = self.conn.cursor()
    def findMovieById(self, id):
        query = "SELECT m.title, \
                c.country_name, \
                m.year_released, \
                m.runtime FROM movies m JOIN countries c ON m.country = c.country_code \
                WHERE m.movieid = %s"
        self.cur.execute(query, (id,))
        result = self.cur.fetchone()
        return result


In [23]:
q = DatabaseManipulation()
q.cur = conn.cursor()
print(q.findMovieById(2))

('Al-mummia', 'Egypt', 1969, 102)
